In [1]:
import tensorflow as tf
from src.robot_reboot.dataset_factory import RobotRebootDataSetFactory
import numpy as np
tf.__version__

'2.4.1'

In [11]:
cnn = tf.keras.models.load_model('model/model_0')
from src.ml.model import get_cnn_model, get_model
#cnn = get_cnn_model((31, 31, 9), n_outputs=16, convolutions=3, optimizer='adam', seed=26)
model = get_model((31, 31, 9), n_outputs=16, convolutions=3, optimizer='adam', seed=26)

In [12]:
model.save_weights('model_w/model_0')

In [13]:
model.load_weights('model_w/model_0')

In [3]:
#ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=20, seed=26, playouts=50)
#v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=3)

In [7]:
rows, cols, layers = 31, 31, 9
state = np.random.random(rows * cols * layers)
state.shape

(8649,)

In [10]:
losses = {
    "v": 'mean_squared_error',
    "p": tf.keras.losses.BinaryCrossentropy()
}

model.compile(loss=losses, optimizer='adam')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 31, 31, 9)]  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 31, 31, 64)   9280        input[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 15, 15, 64)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 15, 15, 64)   65600       max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [ ]:
cnn.fit(ds, epochs=2)

In [11]:
s = state.reshape((1,31,31,9))

model.predict(s)

[array([[-0.14319225]], dtype=float32),
 array([[ 0.10592889,  0.005649  ,  0.1445368 , -0.03024584,  0.18114327,
         -0.00848512, -0.02346749, -0.02155034,  0.07705016, -0.12504463,
         -0.05441467, -0.00786909, -0.06173118,  0.17980537, -0.0223503 ,
          0.06519624]], dtype=float32)]

### Serialize

In [ ]:
sample = tf.train.Example(
    features = tf.train.Features(
        feature = {
            'v': tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
            'p': tf.train.Feature(float_list=tf.train.FloatList(value = p)),
            's': tf.train.Feature(float_list=tf.train.FloatList(value = state))
        }
    )
)

In [ ]:
import numpy as np
with tf.io.TFRecordWriter('test.tfrecord') as f:
   for i in range(600):
        v = np.random.rand()
        p = np.random.randn(16)
        state = np.random.randn(31*31*9)
        sample = tf.train.Example(
            features = tf.train.Features(
                feature = {
                    'v': tf.train.Feature(float_list=tf.train.FloatList(value=[v])),
                    'p': tf.train.Feature(float_list=tf.train.FloatList(value = p)),
                    's': tf.train.Feature(float_list=tf.train.FloatList(value = state))
                }
            )
        )
        f.write(sample.SerializeToString())

### Deserialize

In [ ]:
feature_desc = {
    'v': tf.io.FixedLenFeature([], tf.float32),
    'p': tf.io.VarLenFeature(tf.float32),
    's': tf.io.VarLenFeature(tf.float32)
}

In [ ]:
import numpy as np
for serialized in tf.data.TFRecordDataset(['robot_reboot_11.tfrecord']):
    parsed = tf.io.parse_single_example(serialized, feature_desc)
    st= tf.sparse.to_dense(parsed['s'])
    t = tf.reshape(st, (1, rows, cols, layers))
    
    print(parsed['p'])

In [ ]:
v_t.numpy()

### Train NN

In [15]:
def read_tfrecord(example):
    feature_desc = {
        'v': tf.io.FixedLenFeature([], tf.float32),
        'p': tf.io.VarLenFeature(tf.float32),
        's': tf.io.VarLenFeature(tf.float32)
    }
    parsed = tf.io.parse_single_example(example, feature_desc)
    x = tf.reshape(tf.sparse.to_dense(parsed['s']), (1, rows, cols, layers))
    p = tf.reshape(tf.sparse.to_dense(parsed['p']), (1, 16))
    v = tf.reshape(parsed['v'], (1, 1))
    y = {'v': v, 'p': p}
    return x, y

In [16]:
ds = tf.data.TFRecordDataset(['test.tfrecord'])
ds = ds.map(read_tfrecord)

NameError: in user code:

    <ipython-input-15-17529b3322fb>:8 read_tfrecord  *
        x = tf.reshape(tf.sparse.to_dense(parsed['s']), (1, rows, cols, layers))

    NameError: name 'rows' is not defined


In [ ]:
ds

In [ ]:
cnn.fit(ds)

In [ ]:
p.shape

### Testing factory

In [ ]:
ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=10, seed=26, playouts=100)

In [ ]:
v, p, s = ds_factory.create(locate_robot_close_goal=False)

In [ ]:
p

In [ ]:
v

### Run script locally

In [ ]:
!mkdir -p logs
!python train.py --model ./model/model_0 \
                         --train ./data/train \
                         --validation data \
                         --eval /data \
                         --model_dir ./logs \
                         --epochs 1 \
                         --model_version 1
                        